In [ ]:
from google.colab import files
uploaded = files.upload()  # selecione o arquivo .env

Saving .env to .env


In [ ]:
!pip install -U langchain langchain-community openai python-dotenv PyPDF2 faiss-cpu tiktoken sentence-transformers pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
  # Carrega variáveis do .env
  from dotenv import load_dotenv
  import os

  load_dotenv(".env")  # Certifique-se de que o arquivo .env foi carregado

  # Acessa a chave da variável de ambiente
  api_key = os.getenv("OPENROUTER_API_KEY")

  # Inicializa o modelo de linguagem com segurança
  from langchain.chat_models import ChatOpenAI

  llm = ChatOpenAI(
      openai_api_base="https://openrouter.ai/api/v1",
      openai_api_key=api_key,
      model='deepseek/deepseek-r1-zero:free'
  )

  # Subir o PDF manualmente (se necessário)
  from google.colab import files
  uploaded = files.upload()

  from langchain.document_loaders import PyPDFLoader
  pdf_path = "/content/ppc.pdf"  # ajuste o nome do PDF conforme necessário
  loader = PyPDFLoader(pdf_path)
  documents = loader.load()

  print(f"✅ {len(documents)} páginas carregadas.")

  # Embeddings e indexação com FAISS
  from langchain.embeddings import HuggingFaceEmbeddings
  from langchain.vectorstores import FAISS
  from langchain.text_splitter import RecursiveCharacterTextSplitter

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  docs = text_splitter.split_documents(documents)

  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  vectorstore = FAISS.from_documents(docs, embeddings)

  print("✅ Embeddings gerados e indexados com sucesso!")

  # Criação da cadeia de perguntas e respostas
  from langchain.chains import RetrievalQA

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=vectorstore.as_retriever(),
      return_source_documents=True
  )

  # Teste de consulta
  resposta = qa_chain.invoke({"query": "quantos anos de duração esse curso possui?"})
  print("Resposta:", resposta['result'])

  print("\nFontes utilizadas:")
  for i, doc in enumerate(resposta['source_documents']):
      print(f"\nFonte {i+1}:\n{doc.page_content[:500]}...")

<ipython-input-3-ac32e1313dc2>:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Saving ppc.pdf to ppc.pdf
✅ 100 páginas carregadas.


<ipython-input-3-ac32e1313dc2>:38: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embeddings gerados e indexados com sucesso!
Resposta: \boxed{"O curso possui uma duração de 2 anos letivos, com uma duração mínima de 2 anos e uma duração máxima de 3 anos letivos."}

Fontes utilizadas:

Fonte 1:
vida acadêmica em cursos de pós-graduação Lato Sensu e/ou Stricto Sensu. 
EXTENSÃO: Apoio a ações junto à sociedade qu e permita o compartilha mento do conhecimento 
desenvolvido no curso, seja por meio do ensino ou do desenvolvimento de Projetos Integradores e Trabalho de 
Graduação 
3.5 Requisitos e Formas de Acesso 
O ingresso do aluno se dá pela classificação em pr ocesso seletivo vestibular, realizado em uma única 
fase, com provas dos componentes do núcleo comum do Ensino Médio ou equivale...

Fonte 2:
forma a atender aos objetivos da EPT, de acordo com as funções gerencia is, às demandas sociopolíticas e 
culturais e às relações de atores sociais da escola. 
Em síntese, os conteúdos curriculares são planejados de modo contextualizado a objetivos educacionais 
específi